# NBA GAME PREDICTOR

## Install NBA API

In [372]:
pip install nba_api


## Create Dictionary of NBA Teams and their stats from all games from the 2020/21 - current season

In [373]:
from nba_api.stats.static import teams
team_games = {}
nba_teams = teams.get_teams()
for i in range(len(nba_teams)):
  team_info = nba_teams[i]
  team_games[team_info['id']] = None

In [374]:
team_games.keys()

dict_keys([1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766])

I decided to take the past three seasons and no more past that because I would then have to include the 2019-2020 season, where part of the NBA games played were in the bubble in Orlando.

In [375]:
from nba_api.stats.endpoints import leaguegamefinder
for team_id in team_games:
  gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
  games = gamefinder.get_data_frames()[0]
  team_games[team_id] = games[games.SEASON_ID.str[-4:].isin(['2020', '2021', '2022', '2023'])][['SEASON_ID', 'TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL', 'FG_PCT', 'FT_PCT', 'REB', 'TOV', 'PF' ]]
  team_games[team_id]['MATCHUP'] = team_games[team_id]['MATCHUP'].astype(str).apply(lambda x: x[-3:])
  team_games[team_id].rename(columns={'MATCHUP': 'OPP'}, inplace=True)



In [376]:
for team_id in team_games:
  df = team_games[team_id]
  df = df.drop(df[df['WL'].isna()].index)
  team_games[team_id]['WL'] = team_games[team_id]['WL'].replace({'W': 1, 'L': 0})

In [377]:
df = team_games[1610612763]
X = df[['FG_PCT', 'FT_PCT', 'REB', 'TOV', 'PF']].values
y = df[['WL']].values

In [378]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [379]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## BUILDING PREDICTION MODELS AND TESTING ACCURACY

### Logistic Regression

In [380]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [381]:
y_pred = classifier.predict(X_test)

In [382]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Number of correct predictions: ", (cm[0][0] + cm[1][1]))
print("Number of incorrect predictions: ", (cm[0][1] + cm[1][0]))
print("Accuracy Score: ", accuracy_score(y_test, y_pred))

Number of correct predictions:  57
Number of incorrect predictions:  12
Accuracy Score:  0.8260869565217391


### K-Nearest Neighbours

In [383]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 7, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=7)

In [384]:
y_pred = classifier.predict(X_test)

In [385]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Number of correct predictions: ", (cm[0][0] + cm[1][1]))
print("Number of incorrect predictions: ", (cm[0][1] + cm[1][0]))
print("Accuracy Score: ", accuracy_score(y_test, y_pred))

Number of correct predictions:  57
Number of incorrect predictions:  12
Accuracy Score:  0.8260869565217391


### Kernel Support Vector Machine

In [386]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 42)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(random_state=42)

In [387]:
y_pred = classifier.predict(X_test)

In [388]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Number of correct predictions: ", (cm[0][0] + cm[1][1]))
print("Number of incorrect predictions: ", (cm[0][1] + cm[1][0]))
print("Accuracy Score: ", accuracy_score(y_test, y_pred))

Number of correct predictions:  57
Number of incorrect predictions:  12
Accuracy Score:  0.8260869565217391


### Decision Tree

In [389]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'gini', random_state = 42)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [390]:
y_pred = classifier.predict(X_test)

In [391]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Number of correct predictions: ", (cm[0][0] + cm[1][1]))
print("Number of incorrect predictions: ", (cm[0][1] + cm[1][0]))
print("Accuracy Score: ", accuracy_score(y_test, y_pred))

Number of correct predictions:  55
Number of incorrect predictions:  14
Accuracy Score:  0.7971014492753623


### Random Forest

In [392]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier.fit(X_train, y_train)

<ipython-input-392-8416050ea0fd>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [393]:
y_pred = classifier.predict(X_test)

In [394]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Number of correct predictions: ", (cm[0][0] + cm[1][1]))
print("Number of incorrect predictions: ", (cm[0][1] + cm[1][0]))
print("Accuracy Score: ", accuracy_score(y_test, y_pred))

Number of correct predictions:  57
Number of incorrect predictions:  12
Accuracy Score:  0.8260869565217391


### Naive Bayes

In [395]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [396]:
y_pred = classifier.predict(X_test)

In [397]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Number of correct predictions: ", (cm[0][0] + cm[1][1]))
print("Number of incorrect predictions: ", (cm[0][1] + cm[1][0]))
print("Accuracy Score: ", accuracy_score(y_test, y_pred))

Number of correct predictions:  56
Number of incorrect predictions:  13
Accuracy Score:  0.8115942028985508
